## 논문 요약 LLM 서비스

### 환경설정

In [2]:
!pip install langchain openai chromadb pymupdf tiktoken langchain-community pypdf pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.3 MB/s eta 0:00:00
 

In [12]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from IPython.display import display, Markdown
from pypdf import PdfReader
import requests

In [5]:
load_dotenv("secrets.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key is not None, "OPENAI_API_KEY가 secrets.env에 없습니다."
os.environ["OPENAI_API_KEY"] = openai_api_key

### pdf load

In [13]:
def download_pdf_from_arxiv_url(arxiv_url: str, save_dir: str = ".") -> str:
    # 파일명 추출 (e.g., 2005.11401.pdf)
    filename = arxiv_url.rstrip("/").split("/")[-1]
    if not filename.endswith(".pdf"):
        filename += ".pdf"

    local_path = os.path.join(save_dir, filename)

    # 파일 다운로드
    response = requests.get(arxiv_url)
    response.raise_for_status()

    with open(local_path, "wb") as f:
        f.write(response.content)

    return local_path

In [14]:
arxiv_url = "https://arxiv.org/pdf/2005.11401"
pdf_path = download_pdf_from_arxiv_url(arxiv_url)

reader = PdfReader(pdf_path)
print(f"총 페이지 수: {len(reader.pages)}")

총 페이지 수: 19


In [28]:
text = reader.pages[0].extract_text()

# ✅ 결과 출력
print(f"[Page 1] 텍스트 일부:\n")
print(text[:100])

[Page 1] 텍스트 일부:

Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Alek


### langchain doc으로 변환 & chunk split

In [29]:
from langchain_core.documents import Document
from pypdf import PdfReader

reader = PdfReader("2005.11401.pdf")

documents = []
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text and text.strip():
        documents.append(Document(page_content=text.strip(), metadata={"page": i + 1}))


In [41]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)

split_docs = splitter.split_documents(documents)
print(f"총 chunk 수: {len(split_docs)}")
print(split_docs[7].page_content[:300])

총 chunk 수: 119
and non-parametric memory to the “workhorse of NLP,” i.e. sequence-to-sequence (seq2seq) models.
We endow pre-trained, parametric-memory generation models with a non-parametric memory through
a general-purpose ﬁne-tuning approach which we refer to as retrieval-augmented generation (RAG).
We build RA


In [42]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embedding=embedding)

<ipython-input-42-e88466deb335>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


### RAG

사용자 query로 retriever 검색   
retriever 결과로 RAG에 context 넣어서 QA 수행   

In [51]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# 1. 커스텀 프롬프트 템플릿 정의
template = """
You are a helpful and concise assistant specialized in NLP research.

Answer the following question in a **neutral academic tone** in **ko-KR**

Context:
{context}

Question:
{question}

Answer:"""

In [52]:
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [53]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0),
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt}
)

In [54]:
query = "논문 요약해줘"
response = qa_chain.run(query)
print("🧠 답변:\n", response)

🧠 답변:
 이 논문은 RAG( Retrieval-Augmented Generation) 모델을 제안하며, 이 모델이 정보 검색과 생성 작업에서 어떻게 효과적으로 작동하는지를 다룹니다. RAG는 주어진 입력에 대해 관련 문서를 검색하고, 이를 바탕으로 더 정확하고 구체적인 응답을 생성하는 방식으로 작동합니다. 

연구에서는 RAG 모델이 BART와 비교하여 Jeopardy 질문 생성 작업에서 더 사실적이고 구체적인 응답을 생성함을 보여줍니다. 또한, RAG의 검색 메커니즘이 학습된 정보 검색을 통해 작업 성능을 향상시키는 데 기여함을 입증합니다. 

RAG 모델은 비모수적 메모리 모델의 장점을 활용하여, 테스트 시 지식을 쉽게 업데이트할 수 있는 능력을 가지고 있습니다. 이 논문은 RAG의 성능을 다양한 작업에서 평가하고, 검색의 효과성을 분석하며, 생성의 다양성을 높이는 방법에 대해서도 논의합니다. 

결론적으로, RAG는 정보 검색과 자연어 생성의 통합을 통해 더 나은 성능을 발휘하며, 다양한 자연어 처리 작업에서 유용한 도구로 자리 잡을 수 있음을 시사합니다.


In [56]:
query = "What are the key components of RAG?"
response = qa_chain.run(query)
print("🧠 답변:\n", response)

🧠 답변:
 RAG( Retrieval-Augmented Generation)의 주요 구성 요소는 다음과 같습니다:

1. **정보 검색기(Retriever)**: RAG는 외부 지식 소스(예: 위키피디아)에서 관련 문서를 검색하는 역할을 합니다. 이 검색기는 질문에 대한 답변을 찾기 위해 문서 집합에서 정보를 추출합니다.

2. **생성기(Generator)**: 검색된 문서에서 정보를 바탕으로 자연어로 답변을 생성하는 모델입니다. RAG는 생성기와 검색기를 결합하여, 검색된 정보에 기반하여 더 정확하고 사실적인 답변을 생성할 수 있습니다.

3. **파라메트릭 및 비파라메트릭 메모리**: RAG는 파라메트릭 메모리(모델의 파라미터에 저장된 지식)와 비파라메트릭 메모리(외부 지식 소스에서 검색된 정보)를 결합하여, 보다 풍부하고 다양한 답변을 생성할 수 있도록 합니다.

4. **훈련 방식**: RAG는 검색된 문서에 대한 감독 없이도 훈련될 수 있으며, 이는 다양한 실제 응용 프로그램에서 유용합니다. 

이러한 구성 요소들은 RAG가 개방형 질문 응답 및 정보 생성 작업에서 높은 성능을 발휘할 수 있도록 합니다.
